In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import re
import torch
import json_repair
import pandas as pd
from glob import glob 
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


# 제작한 데이터세 불러오기 
file_list = glob("./data/*.csv")
print(file_list)

df = pd.concat([pd.read_csv(file) for file in file_list])
df.shape

['./data/중고나라_게시물_final_20250322_164822.csv', './data/제품_리뷰글_final_20250322_164822.csv', './data/이벤트_안내글_final_20250322_164822.csv', './data/은행 상담_final_20250315_013253.csv', './data/쇼핑 고객 서비스_final_20250315_013253.csv', './data/보험 상담_final_20250315_013253.csv', './data/모임_동호회_홍보글_final_20250322_164822.csv', './data/강의_세미나_홍보글_final_20250322_164822.csv']


(773, 9)

In [2]:
df.head(2)

,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason,mapping
0,제목: 삼성 갤럭시 S20 울트라 판매합니다 💼\n\n안녕하세요! 삼성 갤럭시 S2...,중고나라_게시물,5,"제공된 데이터는 프롬프트의 모든 요구 사항을 충족합니다. 첫째, 개인정보는 이름(김...",제목: 삼성 갤럭시 S20 울트라 판매합니다 💼\n\n안녕하세요! 삼성 갤럭시 S2...,입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"모든 개인정보가 적절한 placeholder로 비식별화되었으며, 동일한 개인정보는 ...","{'서울 강남구 역삼동': '[LOCATION1]', '우리은행 1002-123-4..."
1,"중고나라 게시물: 🎸중고 기타 판매합니다🎸\n\n안녕하세요, 음악을 사랑하는 이재훈...",중고나라_게시물,5,"데이터는 모든 평가 기준을 완벽히 충족합니다. 개인정보는 이름(이재훈), 연락처(전...","중고나라 게시물: 🎸중고 기타 판매합니다🎸\n\n안녕하세요, 음악을 사랑하는 [PE...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,비식별화 데이터는 모든 개인정보가 적절한 placeholder로 대체되었습니다. 이...,"{'이재훈': '[PERSON1]', '서울 마포구 연남동': '[LOCATION1..."


In [3]:

allowed_types=(
    "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
    "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"))




In [4]:
def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    # STEP 1: 라인 단위 분리
    orig_lines = original_text.splitlines() 
    trans_lines = transformed_text.splitlines()
    # 예시 
    # orig_lines[0]: 상담사: "안녕하세요, 김미영 고객님. 보험 상담을 도와드릴 홍성철입니다. 생일이 1985년 4월 12일로 등록되어 있습니다. 맞으신가요?"
    # trans_lines[0]: 상담사: "안녕하세요, [PERSON1] 고객님. 보험 상담을 도와드릴 [PERSON2]입니다. 생일이 [DATEOFBIRTH1]로 등록되어 있습니다. 맞으신가요?"

    # STEP 2: 각 라인에서 placeholder와 일반 텍스트(literal)를 구분
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}
    # 예시 mapping['김미영'] = '[PERSON1]'

    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        # 예시 
        # parts = re.split(generic_pattern, trans_lines[0])
        # parts = [
        #     '상담사: "안녕하세요, ', 
        #     '[PERSON1]', 
        #     ' 고객님. 보험 상담을 도와드릴 ', 
        #     '[PERSON2]', 
        #     '입니다. 생일이 ', 
        #     '[DATEOFBIRTH1]', 
        #     '로 등록되어 있습니다. 맞으신가요?"'
        # ]

        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping

In [5]:
# OpenAI로 생성할 때 ` 라는 특수문자가 생성되는 경우가 있어서 제거 -> 평가할때 오작동을 일으킵니다. 
df["anonymized_data"] = df["anonymized_data"].map(lambda x: x.replace("`", ""))

# extract_placeholder_mapping를 apply와 함께 사용해서 mapping 컴럼을 추가합니다. 
df["mapping"] = df.apply(lambda x: extract_placeholder_mapping(
    x["origin_data"], 
    x["anonymized_data"], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TWITTER_ID", "TELEGRAM_ID")), 
    axis=1)
df["mapping"] = df["mapping"].map(lambda x:str(x))

In [6]:
# 학습한 모델을 경로를 지정합니다.
save_dir = "./model/model_0.0002_alpha-8_r-16"
peft_model_id = f"{save_dir}"

# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
).to("cuda")

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'  
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
import datasets 

# 데이터셋의 각 샘플을 챗봇 학습에 맞는 포맷으로 변환하는 함수 정의
def get_chat_format(element):
    # 시스템 프롬프트를 정의 (Assistant가 해야할 작업 설명)
    system_prompt = "너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해."

    # 챗봇 메시지 포맷으로 데이터 변환
    return {
        "messages": [
            {"role": "system", "content": system_prompt},                 # 시스템 지시사항
            {"role": "user", "content": element["origin_data"]},          # 원본 데이터 (사용자 입력)
            {"role": "assistant", "content": element["anonymized_data"]}, # 비식별화된 데이터 (모델 출력 예시)
        ], 
        "label": element["mapping"]  # 원본과 비식별화된 데이터 간의 매핑(정답 라벨)
    }

# pandas 데이터프레임(df)을 Hugging Face Dataset 형태로 변환
dataset = datasets.Dataset.from_pandas(df)

# 전체 데이터셋에 get_chat_format 함수를 적용하여 챗봇 학습 데이터로 변환
dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)

# 데이터의 순서를 무작위로 섞음 (시드값을 정해 일관성 유지) -> 다양한 데이터가 있기 때문에 이를 섞기 위해 
dataset = dataset.shuffle(seed=42)

# 전체 데이터셋을 학습용과 평가용 데이터로 나눔 (테스트셋 10%, 학습셋 90%)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

Map:   0%|          | 0/773 [00:00<?, ? examples/s]

In [8]:
print(dataset["test"][6]["messages"][1]["content"])

모임명: 서초구 배드민턴 클럽

안녕하세요! 🏸 배드민턴을 사랑하는 모든 분들을 환영합니다! 저희 '서초구 배드민턴 클럽'은 매주 토요일 오전 10시에 서초구 종합체육관에서 모여 함께 배드민턴을 치며 즐거운 시간을 보내고 있습니다. 초보자부터 고수까지, 누구나 환영하니 부담 없이 오세요!

모임 일시: 매주 토요일 오전 10시
장소: 서초구 종합체육관 (서울시 서초구 서초동 123-45)

가입 방법은 간단해요! 저희 모임의 대표인 김은주에게 연락 주시면 됩니다. 카카오톡 ID: badminton_love로 메시지 남겨주시면 친절히 안내해드리겠습니다. 😊 또는 이메일 badminton_love@naver.com 으로도 문의 가능하니 편하신 방법으로 연락 주세요.

새로운 친구들과 함께 땀 흘리고, 운동 후에는 근처 카페에서 다 같이 이야기를 나누며 친목도 다질 수 있습니다. 체육관 대여비는 회비로 모아 사용하니, 처음 오실 때는 컵라면 정도만 준비해주시면 돼요! 😄

궁금한 점이 있거나 참여하고 싶으신 분들은 언제든지 연락 주세요. 여러분의 많은 참여 기다리고 있겠습니다! 🏸✨

연락처:
- 김은주: 카카오톡 ID badminton_love
- 이메일: badminton_love@naver.com
- 전화번호: 010-8765-4321


In [9]:
print(dataset["test"][6]["messages"][2]["content"])

모임명: [LOCATION1] 배드민턴 클럽

안녕하세요! 🏸 배드민턴을 사랑하는 모든 분들을 환영합니다! 저희 '[LOCATION1] 배드민턴 클럽'은 매주 토요일 오전 10시에 [LOCATION2]에서 모여 함께 배드민턴을 치며 즐거운 시간을 보내고 있습니다. 초보자부터 고수까지, 누구나 환영하니 부담 없이 오세요!

모임 일시: 매주 토요일 오전 10시  
장소: [LOCATION2] ([ADDRESS1])

가입 방법은 간단해요! 저희 모임의 대표인 [PERSON1]에게 연락 주시면 됩니다. 카카오톡 ID: [KAKAO_ID1]로 메시지 남겨주시면 친절히 안내해드리겠습니다. 😊 또는 이메일 [EMAIL1] 으로도 문의 가능하니 편하신 방법으로 연락 주세요.

새로운 친구들과 함께 땀 흘리고, 운동 후에는 근처 카페에서 다 같이 이야기를 나누며 친목도 다질 수 있습니다. 체육관 대여비는 회비로 모아 사용하니, 처음 오실 때는 컵라면 정도만 준비해주시면 돼요! 😄

궁금한 점이 있거나 참여하고 싶으신 분들은 언제든지 연락 주세요. 여러분의 많은 참여 기다리고 있겠습니다! 🏸✨

연락처:
- [PERSON1]: 카카오톡 ID [KAKAO_ID1]
- 이메일: [EMAIL1]
- 전화번호: [CONTACT1]


In [10]:
print(dataset["test"][6]["label"])

{'서초구': '[LOCATION1]', '서초구 종합체육관': '[LOCATION2]', '서울시 서초구 서초동 123-45': '[ADDRESS1]', '김은주': '[PERSON1]', 'badminton_love@naver.com': '[EMAIL1]', '010-8765-4321': '[CONTACT1]'}


In [11]:
# 테스트 데이터셋을 하나씩 순회하면서 평가를 진행
for conv_data in dataset["test"]:
    # tokenizer의 챗 템플릿을 이용하여 입력 메시지(시스템+사용자)를 자연스러운 챗 형식으로 변환
    # tokenize=False로 설정하여 텍스트 형태로 반환
    # add_generation_prompt=True로 설정하여 Assistant가 답변할 준비를 함
    input_data = tokenizer.apply_chat_template(
        conv_data["messages"][:2], tokenize=False, add_generation_prompt=True
    )

    # 변환된 텍스트 데이터를 모델 입력으로 사용하기 위해 토큰화하고 GPU(CUDA)에 로딩
    inputs = tokenizer(input_data, return_tensors="pt").to("cuda")

    # fine-tuned 모델을 사용하여 비식별화된 결과를 생성
    result = fine_tuned_model.generate(
        **inputs,                      # 토큰화된 입력 전달
        max_new_tokens=512,            # 생성할 최대 토큰 개수 설정
        temperature=0.1,               # 낮은 temperature로 예측 결과의 랜덤성을 낮춤 (결과가 일관됨)
        pad_token_id=tokenizer.eos_token_id  # 문장 끝을 나타내는 토큰 설정 (패딩 방지)
    )

    # 생성된 결과에서 입력 부분은 제외하고 새로 생성된 결과(모델의 응답)만 추출하여 텍스트로 변환
    output = tokenizer.decode(
        result[0][len(inputs.input_ids[0]):],  # 입력 부분 토큰을 제외한 생성된 부분만 추출
        skip_special_tokens=True               # 특수 토큰을 제외하고 텍스트 변환
    )

    # 한 번만 실행하고 반복문 종료 (테스트를 위한 예시)
    break

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [12]:
conv_data

{'messages': [{'content': '너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해.',
   'role': 'system'},
  {'content': '🎨 아트 오브 서울 - 현대 미술 동호회 🎨\n\n안녕하세요! 예술을 사랑하는 여러분들을 위한 모임, "아트 오브 서울"에 초대합니다! 저희 모임은 현대 미술을 주제로 다양한 전시회 탐방과 미술작품 감상, 작품 창작 활동을 함께하며, 예술을 매개로 한 소통의 장을 마련하고 있습니다.\n\n🌟 모임 일시: 매달 첫째 주 토요일 오후 3시\n🌟 장소: 서울특별시 종로구 사직로 161, 세종문화회관 아트홀 3층\n🌟 활동 내용: 전시회 탐방, 작품 토론, 아트 창작 워크숍 등\n\n가입 방법은 간단합니다! 아래 연락처로 성함과 간단한 자기소개를 보내주시면 됩니다:\n\n📧 이메일: artinseoul@gmail.com\n📞 전화번호: 010-1234-5678\n📱 카카오톡 ID: artseoul\n\n모임장 박주희가 여러분의 참여를 기다립니다. 미술에 대한 열정과 열린 마음을 가진 여러분과 함께 풍성한 예술의 세계를 탐험하길 기대합니다. 궁금한 사항은 언제든지 연락주세요. 😊\n\n감사합니다! 여러분의 많은 관심과 참여 부탁드립니다! ✨',
   'role': 'user'},
  {'content': '🎨 아트 오브 [LOCATION1] - 현대 미술 동호회 🎨\n\n안녕하세요! 예술을 사랑하는 여러분들을 위한 모임, "아트 오브 [LOCATION1]"에 초대합니다! 저희 모임은 현대 미술을 주제로 다양한 전시회 탐방과 미술작품 감상, 작품 창작 활동을 함께하며, 예술을 매개로 한 소통의 장을 마련하고 있습니다.\n\n🌟 모임 일시: 매달 첫째 주 토요일 오후 3시  \n🌟 장소: [ADDRESS1], 세종문화회관 아트홀 3층  \n🌟 활동 내용: 전시회 탐방, 작품 토론, 아트 창작 워크숍 등\n\n가입 방법은 간단합니다! 아래 

In [13]:
raw_input = conv_data["messages"][1]["content"]
print(raw_input)

🎨 아트 오브 서울 - 현대 미술 동호회 🎨

안녕하세요! 예술을 사랑하는 여러분들을 위한 모임, "아트 오브 서울"에 초대합니다! 저희 모임은 현대 미술을 주제로 다양한 전시회 탐방과 미술작품 감상, 작품 창작 활동을 함께하며, 예술을 매개로 한 소통의 장을 마련하고 있습니다.

🌟 모임 일시: 매달 첫째 주 토요일 오후 3시
🌟 장소: 서울특별시 종로구 사직로 161, 세종문화회관 아트홀 3층
🌟 활동 내용: 전시회 탐방, 작품 토론, 아트 창작 워크숍 등

가입 방법은 간단합니다! 아래 연락처로 성함과 간단한 자기소개를 보내주시면 됩니다:

📧 이메일: artinseoul@gmail.com
📞 전화번호: 010-1234-5678
📱 카카오톡 ID: artseoul

모임장 박주희가 여러분의 참여를 기다립니다. 미술에 대한 열정과 열린 마음을 가진 여러분과 함께 풍성한 예술의 세계를 탐험하길 기대합니다. 궁금한 사항은 언제든지 연락주세요. 😊

감사합니다! 여러분의 많은 관심과 참여 부탁드립니다! ✨


In [14]:
print(output)

🎨 아트 오브 [LOCATION1] - 현대 미술 동호회 🎨

안녕하세요! 예술을 사랑하는 여러분들을 위한 모임, "아트 오브 [LOCATION1]"에 초대합니다! 저희 모임은 현대 미술을 주제로 다양한 전시회 탐방과 미술작품 감상, 작품 창작 활동을 함께하며, 예술을 매개로 한 소통의 장을 마련하고 있습니다.

🌟 모임 일시: 매달 첫째 주 토요일 오후 3시
🌟 장소: [ADDRESS1], [LOCATION2] 아트홀 3층
🌟 활동 내용: 전시회 탐방, 작품 토론, 아트 창작 워크숍 등

가입 방법은 간단합니다! 아래 연락처로 성함과 간단한 자기소개를 보내주시면 됩니다:

📧 이메일: [EMAIL1]
📞 전화번호: [CONTACT1]
📱 카카오톡 ID: [KAKAO_ID1]

모임장 [PERSON1]가 여러분의 참여를 기다립니다. 미술에 대한 열정과 열린 마음을 가진 여러분과 함께 풍성한 예술의 세계를 탐험하길 기대합니다. 궁금한 사항은 언제든지 연락주세요. 😊

감사합니다! 여러분의 많은 관심과 참여 부탁드립니다! ✨


In [15]:
# 원본(raw_input)과 모델의 출력(output)을 비교하여 placeholder 매핑 정보를 추출
mapping_result = extract_placeholder_mapping(
    raw_input,  # 원본 텍스트 (개인정보 포함)
    output,     # 모델이 생성한 텍스트 (개인정보가 비식별화된 placeholder로 대체됨)
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"
    )  # 비식별화에 사용된 placeholder의 유형들 지정
)

# 추출한 매핑 결과(실제 개인정보 ↔ placeholder 대응)를 출력하여 확인
display(mapping_result)

{'서울': '[LOCATION1]',
 '서울특별시 종로구 사직로 161': '[ADDRESS1]',
 '세종문화회관': '[LOCATION2]',
 'artinseoul@gmail.com': '[EMAIL1]',
 '010-1234-5678': '[CONTACT1]',
 '박주희': '[PERSON1]'}

In [16]:
display(json_repair.loads(conv_data["label"]))

{'서울': '[LOCATION1]',
 '서울특별시 종로구 사직로 161, 세종문화회관 아트홀 3층': '[ADDRESS1]',
 'artinseoul@gmail.com': '[EMAIL1]',
 '010-1234-5678': '[CONTACT1]',
 '박주희': '[PERSON1]'}

In [17]:
# 실제 정답 매핑(Ground Truth)과 모델이 예측한 매핑 결과를 비교하여 정확도를 평가하는 함수
def compare_mappings(ground_truth, prediction, consider_partial_match=True):
    # 1. 두 매핑의 키(개인정보 항목)를 비교
    gt_keys = set(ground_truth.keys())  # 정답의 키들
    pred_keys = set(prediction.keys())  # 모델 예측의 키들

    common_keys = gt_keys.intersection(pred_keys)  # 정답과 예측 모두에 존재하는 키
    only_in_gt = gt_keys - pred_keys               # 정답에만 존재하는 키 (누락된 항목)
    only_in_pred = pred_keys - gt_keys             # 예측에만 존재하는 키 (추가된 항목)

    # 2. 완전 일치하는 항목을 계산 (키가 같고 값도 정확히 같아야 함)
    correct_values = sum(1 for k in common_keys if ground_truth[k] == prediction[k])

    # 3. 부분 일치 항목 찾기 (값이 정확히 일치하지만 키가 약간 다른 경우)
    partial_matches = []  # 부분 일치 항목의 쌍 (정답키, 예측키)
    partial_correct = 0   # 부분 일치 개수 카운트

    if consider_partial_match:
        # 정답에만 있는 키를 기준으로 부분적으로 유사한 예측 키가 있는지 확인
        for gt_key in list(only_in_gt):
            for pred_key in list(only_in_pred):
                # 키가 서로 포함관계(부분 문자열 관계)이면 부분 일치로 판단
                if (gt_key in pred_key or pred_key in gt_key):
                    if ground_truth[gt_key] == prediction[pred_key]:
                        partial_matches.append((gt_key, pred_key))
                        partial_correct += 1
                        only_in_gt.remove(gt_key)     # 부분 일치 확인된 키는 목록에서 제거
                        only_in_pred.remove(pred_key) # 부분 일치 확인된 키는 목록에서 제거
                        break

    # 4. 평가 결과 출력 (상세한 분석)
    print(f"완전 일치 키 수: {len(common_keys)}/{len(gt_keys)} ({len(common_keys)/len(gt_keys)*100:.2f}%)")

    if consider_partial_match:
        print(f"부분 일치 키 쌍 수: {len(partial_matches)}/{len(gt_keys)} ({len(partial_matches)/len(gt_keys)*100:.2f}%)")
        print(f"부분 일치 키 쌍: {partial_matches}")

    print(f"GT에만 있는 키: {only_in_gt}")    # 누락된 키 목록
    print(f"예측에만 있는 키: {only_in_pred}") # 추가된 키 목록

    # 5. 정확도 계산 (완전일치 + 부분일치를 포함하여 계산)
    total_correct = correct_values + partial_correct
    accuracy = total_correct / len(gt_keys) if gt_keys else 0
    print(f"전체 정확도(부분 일치 포함): {accuracy:.4f} ({total_correct}/{len(gt_keys)})")

    # 완전 일치만 고려한 엄격한 정확도 계산
    strict_accuracy = correct_values / len(gt_keys) if gt_keys else 0
    print(f"엄격한 정확도(완전 일치만): {strict_accuracy:.4f} ({correct_values}/{len(gt_keys)})")

    # 최종 결과를 dictionary로 반환
    return {
        "accuracy": accuracy,
        "strict_accuracy": strict_accuracy,
        "common_keys": common_keys,
        "correct_values": correct_values,
        "partial_matches": partial_matches,
        "partial_correct": partial_correct,
        "only_in_gt": only_in_gt,
        "only_in_pred": only_in_pred
    }

# 실제 데이터에서 가져온 정답 라벨(json 형태)을 dictionary로 변환
gt_mapping = json_repair.loads(conv_data["label"])

# 모델이 예측한 매핑 결과 사용
pred_mapping = mapping_result

# 두 매핑 결과를 비교하여 정확도 평가 실행
accuracy = compare_mappings(gt_mapping, pred_mapping)

완전 일치 키 수: 4/5 (80.00%)
부분 일치 키 쌍 수: 1/5 (20.00%)
부분 일치 키 쌍: [('서울특별시 종로구 사직로 161, 세종문화회관 아트홀 3층', '서울특별시 종로구 사직로 161')]
GT에만 있는 키: set()
예측에만 있는 키: {'세종문화회관'}
전체 정확도(부분 일치 포함): 1.0000 (5/5)
엄격한 정확도(완전 일치만): 0.8000 (4/5)


In [18]:
def compare_simple_mappings(ground_truth, prediction, sample_idx, consider_partial_match=True):
    eval_result = compare_mappings(ground_truth, prediction, consider_partial_match)
    eval_result["sample_idx"] = sample_idx
    return eval_result


def aggregate_results_simple(results):
    """
    여러 개의 샘플에 대한 개별 평가 결과를 하나로 종합하여 요약하는 함수입니다.
    간결한 형식으로 주요 지표를 집계하여 반환합니다.
    """
    total_samples = len(results)  # 전체 평가 샘플 수

    # 샘플별 정확도와 엄격한 정확도의 평균 계산
    total_accuracy = sum(r["evaluation"]["accuracy"] for r in results) / total_samples
    strict_accuracy = sum(r["evaluation"]["strict_accuracy"] for r in results) / total_samples

    # 부분 일치한 케이스 수집 (샘플 인덱스 포함)
    all_partial_matches = []
    for r in results:
        for gt_key, pred_key in r["evaluation"]["partial_matches"]:
            all_partial_matches.append((r["sample_idx"], gt_key, pred_key))

    # 가장 흔히 발생한 부분 일치 패턴(샘플 번호와 함께)을 집계
    partial_match_patterns = {}
    for idx, gt_key, pred_key in all_partial_matches:
        pattern = f"샘플 {idx}: {gt_key} -> {pred_key}"
        partial_match_patterns[pattern] = partial_match_patterns.get(pattern, 0) + 1

    # 가장 흔히 발생한 오류(추가/누락) 패턴을 샘플 번호와 함께 집계
    error_patterns = {}
    for r in results:
        # 정답에만 있는 키 (누락)
        for gt_key in r["evaluation"]["only_in_gt"]:
            pattern = f"샘플 {r['sample_idx']}: 누락 : {gt_key}"
            error_patterns[pattern] = error_patterns.get(pattern, 0) + 1
        # 예측에만 있는 키 (추가)
        for pred_key in r["evaluation"]["only_in_pred"]:
            pattern = f"샘플 {r['sample_idx']}: 추가 : {pred_key}"
            error_patterns[pattern] = error_patterns.get(pattern, 0) + 1

    # 집계된 결과를 반환
    return {
        "total_samples": total_samples,
        "average_accuracy": total_accuracy,
        "average_strict_accuracy": strict_accuracy,
        "common_partial_matches": sorted(partial_match_patterns.items(), key=lambda x: x[1], reverse=True),
        "common_errors": sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)
    }


def evaluate_anonymization_model_simple(model, tokenizer, test_dataset, num_samples=None, temperature=0.1, max_new_tokens=512):
    """
    모델의 비식별화 성능을 평가하는 함수 (간결한 출력 버전)
    """
    # 평가할 샘플 개수 지정 (None이면 전체 데이터셋 사용)
    if num_samples is None:
        samples = test_dataset
    else:
        samples = test_dataset.select(range(min(num_samples, len(test_dataset))))

    results = []  # 평가 결과를 저장할 리스트

    # 데이터셋을 순회하며 평가
    for idx, conv_data in enumerate(samples):
        print("------------------------------------")
        print(f"샘플 {idx+1}/{len(samples)} 평가 중...")

        # 입력 메시지를 챗 포맷으로 변환하여 준비 (시스템 프롬프트와 사용자 입력만 사용)
        input_data = tokenizer.apply_chat_template(
            conv_data["messages"][:2],
            tokenize=False,
            add_generation_prompt=True
        )

        # 입력 데이터를 토큰화하여 모델에 전달
        inputs = tokenizer(input_data, return_tensors="pt").to(model.device)

        # 모델이 예측을 수행하여 결과 생성
        result = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )

        # 모델의 생성 결과를 텍스트로 변환 (입력 부분은 제외)
        output = tokenizer.decode(result[0][len(inputs.input_ids[0]):], skip_special_tokens=True)

        # 원본 입력 텍스트 추출
        raw_input = conv_data["messages"][1]["content"]

        # 원본과 생성 결과를 비교하여 placeholder 매핑 추출
        mapping_result = extract_placeholder_mapping(
            raw_input.replace("`", ""),
            output.replace("`", ""),
            allowed_types=(
                "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH",
                "EMAIL", "LOCATION", "KAKO_ID", "TWITTER_ID", "TELEGRAM_ID"
            )
        )

        # 정답 매핑 데이터 로드
        gt_mapping = json_repair.loads(conv_data["label"])

        # 모델 예측 결과와 정답을 비교하여 평가 수행
        evaluation = compare_simple_mappings(gt_mapping, mapping_result, sample_idx=idx)

        # 개별 샘플의 평가 결과 저장
        results.append({
            "sample_idx": idx,
            "raw_input": raw_input,
            "model_output": output,
            "ground_truth_mapping": gt_mapping,
            "predicted_mapping": mapping_result,
            "evaluation": evaluation
        })

    # 모든 샘플 평가 결과를 종합하여 요약
    overall_results = aggregate_results_simple(results)

    # 개별 샘플 평가 결과와 종합 평가 결과 반환
    return {
        "sample_results": results,
        "overall_results": overall_results
    }

# 사용 예시
# 전체 테스트 데이터셋을 사용하여 평가 수행
eval_results = evaluate_anonymization_model_simple(fine_tuned_model, tokenizer, dataset["test"], num_samples=len(dataset["test"]))

------------------------------------
샘플 1/78 평가 중...
완전 일치 키 수: 4/5 (80.00%)
부분 일치 키 쌍 수: 1/5 (20.00%)
부분 일치 키 쌍: [('서울특별시 종로구 사직로 161, 세종문화회관 아트홀 3층', '서울특별시 종로구 사직로 161')]
GT에만 있는 키: set()
예측에만 있는 키: {'세종문화회관'}
전체 정확도(부분 일치 포함): 1.0000 (5/5)
엄격한 정확도(완전 일치만): 0.8000 (4/5)
------------------------------------
샘플 2/78 평가 중...
완전 일치 키 수: 7/7 (100.00%)
부분 일치 키 쌍 수: 0/7 (0.00%)
부분 일치 키 쌍: []
GT에만 있는 키: set()
예측에만 있는 키: set()
전체 정확도(부분 일치 포함): 1.0000 (7/7)
엄격한 정확도(완전 일치만): 1.0000 (7/7)
------------------------------------
샘플 3/78 평가 중...
완전 일치 키 수: 7/7 (100.00%)
부분 일치 키 쌍 수: 0/7 (0.00%)
부분 일치 키 쌍: []
GT에만 있는 키: set()
예측에만 있는 키: set()
전체 정확도(부분 일치 포함): 1.0000 (7/7)
엄격한 정확도(완전 일치만): 1.0000 (7/7)
------------------------------------
샘플 4/78 평가 중...
완전 일치 키 수: 5/5 (100.00%)
부분 일치 키 쌍 수: 0/5 (0.00%)
부분 일치 키 쌍: []
GT에만 있는 키: set()
예측에만 있는 키: set()
전체 정확도(부분 일치 포함): 1.0000 (5/5)
엄격한 정확도(완전 일치만): 1.0000 (5/5)
------------------------------------
샘플 5/78 평가 중...
완전 일치 키 수: 7/7 (100.00%)
부분 일치 키 쌍 수:

📌 정확한 의미  
- 누락 (GT에만 있는 키):
    - 정답 데이터(Ground Truth) 에는 존재하는데, 모델의 예측 결과에는 없는 항목입니다.  
    → 모델이 놓쳐서 못 맞춘 항목입니다.

- 추가 (예측에만 있는 키):  
    - 모델의 예측 결과 에는 존재하지만, 정답 데이터(Ground Truth)에는 없는 항목입니다.  
    → 모델이 잘못 예측하여 틀린 항목입니다. (오탐지, 잘못된 추가)

In [19]:
def print_evaluation_summary_comprehensive(eval_results):
    """
    평가 결과 요약을 출력하는 함수 (종합적인 평가 지표 포함)
    """
    overall = eval_results["overall_results"]

    # 전체 데이터셋에 대한 정확도 및 추가 지표 계산을 위한 초기화
    total_gt_keys = 0              # 총 정답 키 개수
    total_pred_keys = 0            # 총 예측 키 개수
    total_correct_keys = 0         # 완전 일치한 키 개수
    total_partial_matches = 0      # 부분 일치한 키 개수

    # 개체 유형별 통계 데이터를 담을 딕셔너리
    entity_type_stats = {}

    # 각 샘플별 평가 결과를 순회하며 통계값 계산
    for r in eval_results["sample_results"]:
        gt_mapping = r["ground_truth_mapping"]
        pred_mapping = r["predicted_mapping"]

        total_gt_keys += len(gt_mapping)
        total_pred_keys += len(pred_mapping)
        total_correct_keys += r["evaluation"]["correct_values"]
        total_partial_matches += r["evaluation"]["partial_correct"]

        # 정답 데이터에서 개체 유형별 통계 수집
        for key, value in gt_mapping.items():
            # 개체 유형 추출 (예시: [PERSON1]에서 PERSON)
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type not in entity_type_stats:
                    entity_type_stats[entity_type] = {"gt": 0, "correct": 0, "partial": 0, "pred": 0}

                entity_type_stats[entity_type]["gt"] += 1

                # 완전 일치 여부 확인
                if key in pred_mapping and pred_mapping[key] == value:
                    entity_type_stats[entity_type]["correct"] += 1

        # 예측 데이터에서 개체 유형별 통계 수집
        for key, value in pred_mapping.items():
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type not in entity_type_stats:
                    entity_type_stats[entity_type] = {"gt": 0, "correct": 0, "partial": 0, "pred": 0}

                entity_type_stats[entity_type]["pred"] += 1

        # 부분 일치된 개체 유형별 통계 수집
        for gt_key, pred_key in r["evaluation"]["partial_matches"]:
            value = gt_mapping[gt_key]
            entity_type = re.match(r'\[([A-Z_]+)', value)
            if entity_type:
                entity_type = entity_type.group(1)
                if entity_type in entity_type_stats:
                    entity_type_stats[entity_type]["partial"] += 1

    # 전체 평가 지표 계산 (정확도, 정밀도, 재현율, F1 점수)
    accuracy = (total_correct_keys + total_partial_matches) / total_gt_keys if total_gt_keys > 0 else 0
    strict_accuracy = total_correct_keys / total_gt_keys if total_gt_keys > 0 else 0

    precision = total_correct_keys / total_pred_keys if total_pred_keys > 0 else 0
    recall = total_correct_keys / total_gt_keys if total_gt_keys > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # 부분 일치를 포함한 유연한 평가 지표 계산
    precision_with_partial = (total_correct_keys + total_partial_matches) / total_pred_keys if total_pred_keys > 0 else 0
    recall_with_partial = (total_correct_keys + total_partial_matches) / total_gt_keys if total_gt_keys > 0 else 0
    f1_with_partial = 2 * precision_with_partial * recall_with_partial / (precision_with_partial + recall_with_partial) if (precision_with_partial + recall_with_partial) > 0 else 0

    # 종합 평가 결과 출력
    print("=" * 70)
    print(f"총 평가 샘플 수: {overall['total_samples']}")

    # 전체 데이터셋 평가 지표 출력
    print("\n전체 데이터셋 평가 지표:")
    print(f"총 정답 키 수: {total_gt_keys}")
    print(f"총 예측 키 수: {total_pred_keys}")
    print(f"완전 일치 키 수: {total_correct_keys} ({total_correct_keys/total_gt_keys*100:.2f}%)")
    print(f"부분 일치 키 수: {total_partial_matches} ({total_partial_matches/total_gt_keys*100:.2f}%)")

    # 엄격한 평가 지표 출력
    print("\n엄격한 평가 (완전 일치만):")
    print(f"정확도(Accuracy): {strict_accuracy:.4f}")
    print(f"정밀도(Precision): {precision:.4f}")
    print(f"재현율(Recall): {recall:.4f}")
    print(f"F1 점수: {f1:.4f}")

    # 유연한 평가 지표 출력
    print("\n유연한 평가 (부분 일치 포함):")
    print(f"정확도(Accuracy): {accuracy:.4f}")
    print(f"정밀도(Precision): {precision_with_partial:.4f}")
    print(f"재현율(Recall): {recall_with_partial:.4f}")
    print(f"F1 점수: {f1_with_partial:.4f}")
    
print_evaluation_summary_comprehensive(eval_results)

총 평가 샘플 수: 78

전체 데이터셋 평가 지표:
총 정답 키 수: 455
총 예측 키 수: 458
완전 일치 키 수: 399 (87.69%)
부분 일치 키 수: 22 (4.84%)

엄격한 평가 (완전 일치만):
정확도(Accuracy): 0.8769
정밀도(Precision): 0.8712
재현율(Recall): 0.8769
F1 점수: 0.8740

유연한 평가 (부분 일치 포함):
정확도(Accuracy): 0.9253
정밀도(Precision): 0.9192
재현율(Recall): 0.9253
F1 점수: 0.9222


In [20]:
print(eval_results["sample_results"][57]["raw_input"])
print("---------------------------------------------")
print(eval_results["sample_results"][57]["model_output"])
print("---------------------------------------------")
print(dataset["test"][57]["messages"][2]["content"])

📷⭐️ ***사진과 나눔의 밤*** ⭐️📷

안녕하세요! 여러분의 일상에 특별한 순간을 더해줄 "사진과 나눔의 밤"에 초대합니다. 📸✨ 이번 모임에서는 모든 참가자들이 각자 가장 아끼는 사진을 공유하며 그 속에 담긴 이야기를 나누고, 간단한 기부 활동을 통해 이웃들에게 따뜻한 마음을 전하는 시간을 가질 예정입니다.

🗓 **모임 일시**: 2023년 11월 15일 (수) 오후 7시  
📍 **장소**: 서울특별시 종로구 세종대로 123 현대갤러리 5층  
☎️ **연락처**: 김지수 010-1234-5678 / 이메일: photoloveevent@gmail.com

🌟 **참여 방법**  
- 참가 신청: 위의 이메일로 본인의 이름과 연락처를 보내주세요!  
- 참가비: 10,000원 (모두 기부됩니다)  
- 입금 계좌: 국민은행 123456-78-901234 (예금주: 김지수)  

모임 당일에는 참가자 전원이 가져온 사진을 통해 추억을 나누고, 나눔의 의미를 함께 되새기는 특별한 프로그램이 준비되어 있습니다. 또한 활동을 통해 모인 기부금은 서울 지역 아동센터에 전달될 예정입니다.  

많은 관심과 참여 부탁드립니다! 😊  
여러분의 따뜻한 마음과 멋진 사진을 기다리고 있겠습니다. 💖

**카카오톡 ID로 문의하기**: @photoevent123  

함께 만들어 갈 소중한 시간, 놓치지 마세요! 🎉✨
---------------------------------------------
📷⭐️ ***사진과 나눔의 밤*** ⭐️📷

안녕하세요! 여러분의 일상에 특별한 순간을 더해줄 "사진과 나눔의 밤"에 초대합니다. 📸✨ 이번 모임에서는 모든 참가자들이 각자 가장 아끼는 사진을 공유하며 그 속에 담긴 이야기를 나누고, 간단한 기부 활동을 통해 이웃들에게 따뜻한 마음을 전하는 시간을 가질 예정입니다.

🗓 **모임 일시**: 2023년 11월 15일 (수) 오후 7시  
📍 **장소**: [LOCATION1] 현대갤러리 5층  
☎️ **연락처**: [PERSO

In [21]:
print(eval_results["sample_results"][70]["raw_input"])
print("---------------------------------------------")
print(eval_results["sample_results"][70]["model_output"])
print("---------------------------------------------")
print(dataset["test"][70]["messages"][2]["content"])

모임명: 힐링 워크숍 🌿

활동 내용: 바쁜 일상에서 벗어나 자연 속에서 힐링을 얻는 워크숍입니다. 요가, 명상, 자연 속 산책을 포함한 다양한 프로그램으로 내면의 평화를 찾는 시간을 가집니다. 참가자들은 각자의 힐링 경험을 나누고, 전문 강사의 지도를 통해 몸과 마음의 균형을 맞추게 됩니다.

모임 일시: 2023년 12월 10일 일요일, 오전 10시부터 오후 4시까지

장소: 경기도 가평군 청평면 산골길 123-45, 힐링 하우스

가입 방법: 참가를 원하는 분들은 이메일로 신청서를 보내주세요. 참가비는 계좌 이체로 가능하며, 참가 확정 후에는 세부 일정을 보내드립니다.

연락처:
- 이메일: healingworkshop@gamil.com
- 전화번호: 010-1234-5678
- 참가비 입금 계좌: 국민은행 123-01-987654

모임 안내자: 이수정 (카카오톡 ID: healingmaster)

이번 워크숍은 내면의 안정을 찾고 싶은 모든 분들께 열려 있습니다. 자연과 호흡하며 마음을 정화할 수 있는 기회를 놓치지 마세요! 이번 주에 함께 힐링의 시간을 가져보세요. 🌱
---------------------------------------------
모임명: 힐링 워크숍 🌿

활동 내용: 바쁜 일상에서 벗어나 자연 속에서 힐링을 얻는 워크숍입니다. 요가, 명상, 자연 속 산책을 포함한 다양한 프로그램으로 내면의 평화를 찾는 시간을 가집니다. 참가자들은 각자의 힐링 경험을 나누고, 전문 강사의 지도를 통해 몸과 마음의 균형을 맞추게 됩니다.

모임 일시: 2023년 12월 10일 일요일, 오전 10시부터 오후 4시까지

장소: [LOCATION1], 힐링 하우스

가입 방법: 참가를 원하는 분들은 이메일로 신청서를 보내주세요. 참가비는 계좌 이체로 가능하며, 참가 확정 후에는 세부 일정을 보내드립니다.

연락처:
- 이메일: [EMAIL1]
- 전화번호: [CONTACT1]
- 참가비 입금 계좌: [ACCOUNT1]

모임 안내자: [PERSON1] 

In [29]:
import re
import os
import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv("./credit-env")
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))


class OutputFormat(BaseModel):
    score: int  # 데이터 품질 평가 점수 (1~5)
    reason: str  # 해당 점수를 매긴 이유 (한국어로 설명)


def create_evaluation_prompt(original_text, ground_truth_anonymized, model_prediction):
    """
    LLM이 비식별화 성능을 평가하기 위한 프롬프트를 생성합니다.
    
    Args:
        original_text (str): 원본 텍스트
        ground_truth_anonymized (str): 정답 비식별화 텍스트
        model_prediction (str): 모델이 예측한 비식별화 텍스트
    
    Returns:
        str: 평가 프롬프트
    """
    
    prompt = f"""당신은 개인정보 비식별화 성능을 평가하는 전문가입니다. 원본 텍스트, 정답 비식별화 텍스트, 모델이 예측한 비식별화 텍스트를 분석하여 모델의 성능을 평가해주세요.

# 평가 데이터
## 원본 텍스트:
{original_text}

## 정답 비식별화 텍스트:
{ground_truth_anonymized}

## 모델 예측 비식별화 텍스트:
{model_prediction}

# 평가 지침
1. 개인정보란, 사람 이름, 연락처 (전화번호, 이메일, 카카오톡 ID 등), 주소(거주지), 계좌번호, 소셜미디어 ID (트위터, 텔레그램 등) 등을 지칭한다.  
2. 원본 텍스트에서 개인정보(이름, 주소, 연락처, 계좌번호, 이메일 등)를 모두 식별하세요.
3. 모델이 예측한 비식별화 텍스트에서 개인정보를 모두 식별하세요.
4. 정답 비식별화 텍스트와 모델이 예측한 비식별화 텍스트를 비교하여 모델이 모든 개인정보를 비식별화했는지를 평가하세요. 이때 원본 데이터가 동호회나 모임 등의 이름은 개인정보 했던 내용은 포함하지 마세요.
5. 사람 이름이 2명 등장 할때 정답은 홍길동을 PERSON1로 김철수를 PERSON2로 비식별화 하였는데, 모델을 홍길동을 PERSON2로 김철수를 PERSON1로 비식별화한 것은 문제가 되지 않습니다. 

5점 : 모델이 모든 개인정보를 비식별화했고, 정답 비식별화 텍스트와 모델이 예측한 비식별화 텍스트가 동일합니다.
4점 : 모델이 모든 개인정보를 비식별화한 경우
1점 : 모델이 개인정보를 비식별화하지 못한 경우

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요.
"""
    response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 주어진 데이터를 평가하는 AI입니다. "},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0,
    response_format=OutputFormat
    )
    
    return response

In [30]:
from tqdm.auto import tqdm 

openai_result_list = []
for idx in tqdm(range(len(dataset["test"]))):
    openai_result = create_evaluation_prompt(
        eval_results["sample_results"][idx]["raw_input"], 
        dataset["test"][idx]["messages"][2]["content"], 
        eval_results["sample_results"][idx]["model_output"]
        )
    openai_result_list.append(openai_result.choices[0].message.content)

print("모든 데이터를 openai에 평가하였습니다.")

  0%|          | 0/78 [00:00<?, ?it/s]

모든 데이터를 openai에 평가하였습니다.


In [31]:
openai_result_list[0]

'{"score":4,"reason":"모델은 모든 개인정보를 비식별화하였으나, \'세종문화회관\'을 \'[LOCATION2]\'로 비식별화하여 정답 비식별화 텍스트와 일치하지 않습니다. 따라서 4점을 부여합니다."}'

In [32]:
import pandas as pd 

openai_df = pd.DataFrame(json_repair.loads(temp_data) for temp_data in openai_result_list)
openai_df.head(2)

,score,reason
0,4,"모델은 모든 개인정보를 비식별화하였으나, '세종문화회관'을 '[LOCATION2]'..."
1,5,"모델이 예측한 비식별화 텍스트는 정답 비식별화 텍스트와 완전히 동일하며, 모든 개인..."


In [33]:
openai_df["score"].value_counts()

score
5    44
4    34
Name: count, dtype: int64

- 개인정보라는 것은 정말 중요한 문제이기 때문에 정확하게 예측하는 것이 무척 중요합니다. 
- OpenAI API로 채점을 진행해본 결과, 모든 테스트 데이터셋을 다 맞췄다는 것을 볼 수 있습니다. 